In [7]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pylab as plt
from ipdb import set_trace

with open('.env') as f:
    _data_pth_ = f.readline().strip()
    _data_pth_ = os.path.expanduser(_data_pth_)

### convert nominal

In [8]:
def transform_categorical(dat: pd.DataFrame):
    for c in dat.columns:
        # check categorical
        if not len(dat[[c]].std()):
            dat[[c]] = dat[[c]].fillna(dat[[c]].mode()) # fill na with the mode category
            uniques = dict([(v, i) for i, v in enumerate(set(dat[c]))])
            dat[[c]] = dat[[c]].replace(uniques)

### normalize numerical: clip outliers, fill nan, standardize

In [9]:
def normalize(dat: pd.DataFrame):
    for c in dat.columns:
        # check numerical
        if len(dat[[c]].std()):
            dat[[c]] = dat[[c]].fillna(0)
            _mean, _std = dat[[c]].mean(axis=0), dat[[c]].std(axis=0)
            # clip outliers (3 std away)
            lo, hi = _mean - 3*_std, _mean + 3*_std
            dat[[c]] = dat[[c]].clip(lo.item(), hi.item())
            # normalize
            dat[[c]] = (dat[[c]] - _mean) / _std

### preprocess and save

In [10]:
if not os.path.exists(f'{_data_pth_}/processed/'):
    os.mkdir(f'{_data_pth_}/processed/')
    
for x in ['train_transaction.csv', 'test_transaction.csv', 'test_identity.csv','train_identity.csv', ]:
    if not os.path.exists(f'{_data_pth_}/processed/{x}'):
        
        mat = pd.read_csv(f'{_data_pth_}/{x}')
        # remove ID
        if 'TransactionID' in mat.columns:
            _ids = mat['TransactionID']
            mat.drop(columns=['TransactionID'], inplace=True)
        if 'isFraud' in mat.columns:
            _labels = mat['isFraud']
            mat.drop(columns=['isFraud'], inplace=True)
        transform_categorical(mat)
        normalize(mat)
        mat = pd.concat((_ids, _labels, mat), axis=1)
#         set_trace()
        mat.to_csv(f'{_data_pth_}/processed/{x}', index=False)
        del mat

C:\Users\LIVELO~1\AppData\Local\Temp/ipykernel_6356/2967106877.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  if not len(dat[[c]].std()):


> c:\users\livelo~1\appdata\local\temp\ipykernel_6356\3138857347.py(19)<module>()

ipdb> n
> c:\users\livelo~1\appdata\local\temp\ipykernel_6356\3138857347.py(20)<module>()

ipdb> n
> c:\users\livelo~1\appdata\local\temp\ipykernel_6356\3138857347.py(4)<module>()

ipdb> n
> c:\users\livelo~1\appdata\local\temp\ipykernel_6356\3138857347.py(5)<module>()

ipdb> n
> c:\users\livelo~1\appdata\local\temp\ipykernel_6356\3138857347.py(7)<module>()

ipdb> n
> c:\users\livelo~1\appdata\local\temp\ipykernel_6356\3138857347.py(9)<module>()

ipdb> c
> c:\users\livelo~1\appdata\local\temp\ipykernel_6356\3138857347.py(19)<module>()

ipdb> c
